# 프로젝트 진행 추천 API 테스트

FastAPI의 `/api/PJA/recommend/generate` 엔드포인트를 테스트합니다.

## 테스트 절차
1. 서버 연결 확인
2. 두 가지 테스트 케이스 실행
3. 응답 데이터 구조 검증

In [3]:
import requests
import json
from datetime import datetime

In [4]:
# API 엔드포인트 설정
API_URL = "http://localhost:8000/api/PJA/recommend/generate"

In [6]:
# 테스트용 데이터 생성 (JSON 문자열로 변환)
import json

# 먼저 딕셔너리로 구조를 만들고
project_data = {
    "workspaceId": 1,
    "recommendedCategories": [
        {
            "categoryId": 3,
            "name": "회원 관리 기능",
            "features": [
                {
                    "featureId": 10,
                    "name": "회원 가입",
                    "actions": [
                        {
                            "name": "이메일 인증 로직 구현",
                            "importance": 3,
                            "startDate": "LocalDateTime형식의 날짜",
                            "endDate": "LocalDateTime형식의 날짜"
                        },
                        {
                            "name": "회원 정보 유효성 검증",
                            "importance": 2,
                            "startDate": "LocalDateTime형식의 날짜",
                            "endDate": "LocalDateTime형식의 날짜"
                        }
                    ]
                }
            ]
        }
    ]
}

# JSON 문자열로 변환해서 전송
test_data = {
    "project_list": json.dumps(project_data, ensure_ascii=False),  # 딕셔너리를 JSON 문자열로 변환
    "max_tokens": 3000,
    "temperature": 0.3,
    "model": "ft:gpt-4o-mini-2024-07-18:test::BebIPMSD"
}

# API 요청 보내기
response = requests.post(API_URL, json=test_data)

# 응답 확인
print("Status Code:", response.status_code)
print("\nResponse:")
print(json.dumps(response.json(), indent=2, ensure_ascii=False))

Status Code: 500

Response:
{
  "detail": "추천 생성 오류: 1 validation error for RecommendationResponse\nrecommendations\n  Input should be a valid list [type=list_type, input_value={'workspaceId': 1, 'categ...'2024-01-20T17:00:00'}]}, input_type=dict]\n    For further information visit https://errors.pydantic.dev/2.11/v/list_type"
}


In [4]:
# 다른 테스트 케이스
test_data_2 = {
    "project_list": "프론트엔드 UI 개발 진행 중. 로그인 페이지 디자인 완료. 회원가입 폼 유효성 검증 필요.",
    "max_tokens": 3000,
    "temperature": 0.3,
    "model": "ft:gpt-4o-mini-2024-07-18:test::BebIPMSD"
}

response = requests.post(API_URL, json=test_data_2)
print("Status Code:", response.status_code)
print("\nResponse:")
print(json.dumps(response.json(), indent=2, ensure_ascii=False))

Status Code: 500

Response:
{
  "detail": "추천 생성 오류: 1 validation error for RecommendationResponse\nrecommendations\n  Input should be a valid list [type=list_type, input_value={'workspaceId': 1, 'categ...'2024-01-20T18:00:00'}]}, input_type=dict]\n    For further information visit https://errors.pydantic.dev/2.11/v/list_type"
}


## 응답 형식 검증

받은 응답이 예상한 형식과 일치하는지 확인

In [ ]:
def validate_response(response_data):
    """응답 데이터의 구조를 검증하는 함수"""
    required_fields = [
        'recommendations',
        'model',
        'total_tokens',
        'prompt_tokens',
        'completion_tokens'
    ]
    
    # 필수 필드 확인
    for field in required_fields:
        if field not in response_data:
            print(f"Missing required field: {field}")
            return False
    
    # recommendations 구조 확인
    recommendations = response_data.get('recommendations', {})
    required_rec_fields = ['workspaceId', 'categoryId', 'featureId', 'recommendedActions']
    
    for field in required_rec_fields:
        if field not in recommendations:
            print(f"Missing field in recommendations: {field}")
            return False
    
    # recommendedActions 구조 확인
    for action in recommendations['recommendedActions']:
        action_fields = ['name', 'importance', 'startDate', 'endDate']
        for field in action_fields:
            if field not in action:
                print(f"Missing field in action: {field}")
                return False
    
    print("Response structure is valid!")
    return True

# 첫 번째 응답 검증
print("Validating first response:")
validate_response(response.json())